In [2]:
%run "./Setuper.ipynb"

In [3]:
from tqdm.notebook import trange
import random
import sys
import numpy as np
import chipwhisperer as cw

class ANN_Observer:
    project = None
    epochs = 0
    n_inputs = 7
    waves = None
    project_path = None
    prev_rand_inputs = None
    akt_rand_inputs = None
    out_data = None   
    cw_setuper : CW_Setuper = None
    trace_len = 24431
    akt_out_data = None
    output_database = []
    
    def __init__(self, _name, _epochs = 300, _cw_setuper : CW_Setuper = None, create = False, targ_periodic_reset = False):
        self.targ_periodic_reset = targ_periodic_reset
        self.epochs = _epochs
        self.akt_rand_inputs = None
        if _cw_setuper is None:
            self.cw_setuper = CW_Setuper(firmware_reinit = True)
        try:
            if create:
                self.load_project(_name = f"projects/{_name}")
                print("✔️ Project succesfully loaded.")
            else:
                raise Exception("Choosen aproach was to create new project, sorry..") 
        except:
            print("⚠️ The mentioned lab was not found!!!")
            print("🕓 Initiating self-creation")
            self.create_project(project_name = _name)
            print("✔️ Project succesfully created.")
        
    def ___create_project(self, project_name = "lab1"): #create_X_traces_with_rand_Nth_bit
        # Create an repositary/save point/project, and get ready chipwhisperer
        project = cw.create_project(f"projects/{project_name}", overwrite = True)
        self.project = project
        try:
            for i in trange(self.n_inputs, desc='Initiating creation of first layer sets of traces'):
                self.measure_traces(bnum = 8, input_size = self.n_inputs, rand_indx = i, rand_input = self.akt_rand_inputs)
            self.measure_traces(bnum = 8, input_size = self.n_inputs, rand_indx = 0, rand_input = np.zeros(self.epochs), change_rand = False)
        except Exception as e:
            print("❌ Failed measurements because: ",e)
            print("⚠️ Proceeding to disconect!!!")
            self._scope_disc_()

    def create_project(self, project_name = "lab1"): #create_X_traces_with_rand_Nth_bit
        # Create an repositary/save point/project, and get ready chipwhisperer
        try:
            project = cw.create_project(f"projects/{project_name}", overwrite = True)
            self.project = project
            self.measure_traces(bnum = 8, input_size = self.n_inputs, rand_indx = 0, rand_input = self.akt_rand_inputs)
        except Exception as e:
            print("❌ Failed measurements because: ",e)
            print("⚠️ Proceeding to disconect!!!")
            self._scope_disc_()
        
    def load_project(self, _name = "./trace.cwp", load = True):
        self.project = cw.open_project(f'projects/{_name}.cwp')
        return self.project

    def _scope_disc_(self):
        try:
            self.cw_setuper.scope.dis()
            self.cw_setuper.target.dis()
            print("✔️ Disconection succesfull.")
        except Exception as e:
            print("❌ Disconection failed because: ",e)
    
    def scope_setup(self, scope, samples=24431, decimate=2):
        # arm the scope
        scope.arm()
        # Set the maximum number of points in a trace
        scope.adc.fifo_fill_mode = "normal"
        scope.adc.samples = samples
        scope.adc.decimate = decimate

    def measure_traces(self, bnum = 8, input_size = 8, rand_indx = 0, rand_input = None, change_rand = False): #create_X_traces_with_rand_Nth_bit 
        #print(f"✔️ Starting measurement for {rand_indx}-th weights..")
        scope = self.cw_setuper.scope
        target = self.cw_setuper.target
        project = self.project
        reset_target(scope) # flush chipwhisperer and re-assing its parameters (this is moot, if used >1x)
        rand_inputs = rand_input
        if rand_input is None:
            print(f"⚠️ Generating rand_input..")
            L = range((2**bnum)-1) # Range of generated rand_numbers
            rand_inputs = [random.choice(L) for _ in range(self.epochs)]
            self.akt_rand_inputs = rand_inputs
        input_array = [np.uint8(0) for i in range(input_size)]
        
        # capture X (epochs) traces, while each has different rand_input
        for i in trange(self.epochs, desc='                                Capturing traces'):
            input_array[rand_indx] = np.uint8(rand_inputs[i])
            # print(input_array)
            self.scope_setup(scope, samples=self.trace_len, decimate=2)
            # flush the UART buffer, normaly it is automatized, but in this case it is called manualy, it in essence flushes buffer storing incoming data t odevice
            target.flush()
            input_data = bytearray(input_array)
            #print(f"Hello here is na input data: {input_data}")
            target.send_cmd('p', 0x80, input_data)
            ret = scope.capture()
            wave = scope.get_last_trace()
            out_data = target.read_cmd('r')
            self.output_database.append(out_data)
            if self.akt_out_data is None:
                self.akt_out_data = out_data
            out_data = None
            ack = target.read_cmd('e')
            trace = cw.Trace(wave, input_data, out_data, [])
            #print(f"Hello here is na output data: {out_data}")
            if trace is None:
                print("None ...")
                returned_data
            project.traces.append(trace)
        
            #Send every 50th trace refress
            if self.targ_periodic_reset:
                if i % 60 == 0:
                    reset_target(scope)
        if change_rand:
            self.prev_rand_inputs = self.akt_rand_inputs
            self.akt_rand_inputs = akt_rand_inputs
        project.save()
        self.project = project
        return project, out_data

    def normalize_trace(self, wave, treshold =  0.15):
        lenght =  len(wave)
        new_wave = np.ones((lenght,), dtype=int)
        for indx  in range(lenght):
            if abs(wave[indx]) < treshold:
                new_wave[indx] = 0
        return new_wave
    # wave  = normalize_trace(project.traces[0][0], treshold = 0.05)
    # plotSaveWave(wave = wave, name_wave = "temp_wave")
    
    def uppSampleArray(self, origin_row, degree): # uppsample by  padding after every time sample
        exp_array = []
        for i in origin_row:
            exp_array.append(i)
            for ii in range (degree):
                exp_array.append(0)
        return exp_array
        
    def createDiffWave(self):
        project = self.project
        waves = []
        diff_waves = []
        square_waves = []
        
        for trace in project.traces:
            waves.append(trace[0])
        avg_wave = np.mean(waves,axis=0)
        
        for i in waves:
            diff_waves.append(np.subtract(i,avg_wave))
        for i in diff_waves:
            square_wave = []
            for ii in i:
                square_wave.append(ii**2)
            square_waves.append(square_wave)
        
        diff_avg = np.mean(diff_waves,axis=0)
        var = np.mean(square_waves,axis=0)
    
        
        return diff_avg, avg_wave, var, waves